In [1]:
import os, sys
from os.path import expanduser
import matplotlib.pyplot as plt
%matplotlib inline

user_home = expanduser("~")
neuron_home = '%s/bin/nrnpython/lib/python' % user_home
model_home = '%s/src/model' % user_home
dentate_home = '%s/dentate' % model_home
results_prefix = os.getenv('DG_RESULTS_DIR', '%s/results' % dentate_home)
sys.path.append(neuron_home) 
sys.path.append(model_home) 

import dentate
from dentate import utils, plot
from dentate.env import Env


/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module '_yaml' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
rel_spike_events_paths = ['', 
                          ]
spike_events_paths = [ '%s/%s' % (results_prefix, spike_events_path) for spike_events_path in rel_spike_events_paths ]
trajectory_path = '%s/datasets/DG_PP_spiketrains_A_20190512.h5' % dentate_home



In [ ]:
    env = None
    if config_path is not None:
        env = Env(config_file=config_path)
        if env.analysis_config is not None:
            baks_config.update(env.analysis_config['Firing Rate Inference'])
            pf_config.update(env.analysis_config['Place Fields'])

    n_inputs = len(spike_input_paths)
    assert(n_inputs > 1)
            
    if len(spike_namespace_ids) == 1:
        spike_namespace_id = spike_namespace_ids[0]
        spike_namespace_ids = [spike_namespace_id]*n_inputs

    assert(len(spike_namespace_ids) == n_inputs)
    assert(len(arena_ids) == n_inputs)
            
    trajectories = [stimulus.read_trajectory(trajectory_path, arena_id, trajectory_id) for arena_id in arena_ids]

    (population_ranges, N) = read_population_ranges(spike_input_paths[0])
    population_names = read_population_names(spike_input_paths[0])

    pop_num_cells = {}
    for k in population_names:
        pop_num_cells[k] = population_ranges[k][1]

    if populations is None:
        populations = list(population_names)

    if output_file_path is not None and not os.path.isfile(output_file_path):
        input_file = h5py.File(spike_input_path, 'r')
        output_file = h5py.File(output_file_path, 'w')
        input_file.copy('/H5Types', output_file)
        input_file.close()
        output_file.close()

    PF_arena_dicts = defaultdict(lambda: defaultdict(lambda: None))
    for spike_input_path, spike_namespace_id, arena_id in zip_longest(spike_input_paths, spike_namespace_ids, arena_ids):
        
        this_spike_namespace = '%s %s %s' % (spike_namespace_id, arena_id, trajectory_id)
        this_spike_namespace = spike_namespace_id
    
        spkdata = spikedata.read_spike_events(spike_input_path, populations, this_spike_namespace,
                                              spike_train_attr_name=spike_train_attr_name, time_range=time_range)

        spkpoplst = spkdata['spkpoplst']
        spkindlst = spkdata['spkindlst']
        spktlst = spkdata['spktlst']
        num_cell_spks = spkdata['num_cell_spks']
        pop_active_cells = spkdata['pop_active_cells']
        tmin = spkdata['tmin']
        tmax = spkdata['tmax']

        if time_range is None:
            time_range = [tmin, tmax]
        time_bins  = np.arange(time_range[0], time_range[1], bin_size)

        for i, subset in enumerate(spkpoplst):

            spkts         = spktlst[i]
            spkinds       = spkindlst[i]
            spkdict       = spikedata.make_spike_dict(spkinds, spkts)

            rate_bin_dict = spikedata.spike_density_estimate(subset, spkdict, time_bins, arena_id=arena_id,
                                                             trajectory_id=trajectory_id,
                                                             output_file_path=output_file_path, **baks_config)
            PF_dict = spikedata.place_fields(subset, bin_size, rate_bin_dict, trajectory, arena_id=arena_id,
                                             trajectory_id=trajectory_id, output_file_path=output_file_path,
                                             **pf_config)
        
            PF_arena_dicts[arena_id][subset] = PF_dict

    PF_corrs_dict = defaultdict(list)
    rest = arena_ids
    while len(rest) > 1:
        arena_id, rest = rest[0], rest[1:]
        for population in populations:
            if population in PF_arena_dicts[arena_id]:
                pf_corr = spikedata.place_field_corr(PF_arena_dicts[arena_id][population], PF_arena_dicts[rest[0]][population])
                PF_corrs_dict[population].append(pf_corr)

    print(PF_corrs_dict)
